# Goal

* Primer design for clade of interest

# Var

In [4]:
base_dir = '/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/'
clade = 'Butyricimonas'
domain = 'Bacteria'
taxid = 574697 

# Init

In [5]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(LeyLabRMisc)

In [6]:
df.dims()
work_dir = file.path(base_dir, clade)
make_dir(work_dir)

Created directory: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Butyricimonas 


# Genome download

* Downloading genomes from NCBI

```
OUTDIR=/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/
mkdir -p $OUTDIR
ncbi-genome-download -p 12 -s genbank -F fasta --genera Butyricimonas -o $OUTDIR bacteria
```

# Genome quality

* Filtering genomes by quality

In [7]:
D = file.path(base_dir, clade, 'genbank')
files = list_files(D, '.fna.gz')
samps = data.frame(Name = files %>% as.character %>% basename,
                   Fasta = files,
                   Domain = 'Bacteria',
                   Taxid = taxid) %>%
    mutate(Name = gsub('\\.fna\\.gz$', '', Name),
           Fasta = gsub('/+', '/', Fasta))
samps

# writing file
outfile = file.path(D, 'samples.txt')
write_table(samps, outfile)

Name,Fasta,Domain,Taxid
<chr>,<chr>,<fct>,<dbl>
GCA_000379665.1_ASM37966v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/genbank/bacteria/GCA_000379665.1/GCA_000379665.1_ASM37966v1_genomic.fna.gz,Bacteria,574697
GCA_000519105.1_ASM51910v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/genbank/bacteria/GCA_000519105.1/GCA_000519105.1_ASM51910v1_genomic.fna.gz,Bacteria,574697
⋮,⋮,⋮,⋮
GCA_902364335.1_MGYG-HGUT-00203_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/genbank/bacteria/GCA_902364335.1/GCA_902364335.1_MGYG-HGUT-00203_genomic.fna.gz,Bacteria,574697
GCA_902376195.1_MGYG-HGUT-01552_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/genbank/bacteria/GCA_902376195.1/GCA_902376195.1_MGYG-HGUT-01552_genomic.fna.gz,Bacteria,574697


File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Butyricimonas/genbank/samples.txt 


### LLG

#### Config

In [8]:
cat_file(file.path(work_dir, 'LLG', 'config.yaml'))

# table with genome --> fasta_file information
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Butyricimonas/genbank/samples.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

# batch processing of genomes for certain steps
## increase to better parallelize
batches: 2 

# Domain of genomes ('Archaea' or 'Bacteria)
## Use "Skip" if provided as a "Domain" column in the genome table
Domain: Skip

# software parameters
# Use "Skip" to skip any of these steps. If no params for rule, use ""
# dRep MAGs are not further analyzed, but you can de-rep & then use the de-rep genome table as input.
params:
  ionice: -c 3
  # assembly assessment
  seqkit: ""
  quast: Skip #""
  multiqc_on_quast: "" 
  checkm: ""
  # de-replication (requires checkm)
  drep: -comp 

#### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llg
$ screen -L -S llg ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/config.yaml 20 -F
```

### Samples table of high-quality genomes

In [19]:
# checkM summary
checkm = file.path(work_dir, 'LLG', 'checkM', 'checkm_qa_summary.tsv') %>%
    read.delim(sep='\t') 
checkm

Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,Strain.heterogeneity,Genome.size..bp.,X..ambiguous.bases,⋯,X0,X1,X2,X3,X4,X5.,assembly.Id,assembler.Id,taxon.Id,File
<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<lgl>,<fct>
GCA_000379665.1_ASM37966v1_genomic,k__Bacteria (UID2569),434,278,186,100.00,0.00,0,4770838,131,⋯,0,278,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Butyricimonas|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.1,markers_qa_summary.tsv.1,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Butyricimonas/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.1
GCA_001915615.1_ASM191561v1_genomic,k__Bacteria (UID2569),434,278,186,88.69,0.27,0,4167409,2477,⋯,44,233,1,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Butyricimonas|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.2,markers_qa_summary.tsv.2,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Butyricimonas/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_900184685.1_PRJEB20857_genomic,k__Bacteria (UID2569),434,278,186,100.00,0,0,5300003,0,⋯,0,278,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Butyricimonas|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.13,markers_qa_summary.tsv.13,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Butyricimonas/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.13
GCA_902364335.1_MGYG-HGUT-00203_genomic,k__Bacteria (UID2569),434,278,186,99.46,0,0,4537596,250,⋯,1,277,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Butyricimonas|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.14,markers_qa_summary.tsv.14,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Butyricimonas/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.14


In [20]:
# dRep summary
drep = file.path(work_dir, 'LLG', 'drep', 'checkm_markers_qa_summary.tsv') %>%
    read.delim(sep='\t') %>%
    mutate(Bin.Id = gsub('.+/', '', genome),
           Bin.Id = gsub('\\.fna$', '', Bin.Id))
drep

genome,completeness,contamination,Bin.Id
<fct>,<dbl>,<dbl>,<chr>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000379665.1_ASM37966v1_genomic.fna,100.00,0.00,GCA_000379665.1_ASM37966v1_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_001915615.1_ASM191561v1_genomic.fna,88.69,0.27,GCA_001915615.1_ASM191561v1_genomic
⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900184685.1_PRJEB20857_genomic.fna,100.00,0,GCA_900184685.1_PRJEB20857_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902364335.1_MGYG-HGUT-00203_genomic.fna,99.46,0,GCA_902364335.1_MGYG-HGUT-00203_genomic


In [21]:
# de-replicated genomes
drep_gen = file.path(work_dir, 'LLG', 'drep', 'dereplicated_genomes.tsv') %>%
    read.delim(sep='\t')
drep_gen

Name,Fasta
<fct>,<fct>
GCA_003991565.1_ASM399156v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/drep/drep/dereplicated_genomes/GCA_003991565.1_ASM399156v1_genomic.fna
GCA_900258545.1_PRJEB22910_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/drep/drep/dereplicated_genomes/GCA_900258545.1_PRJEB22910_genomic.fna
⋮,⋮
GCA_014287455.1_ASM1428745v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/drep/drep/dereplicated_genomes/GCA_014287455.1_ASM1428745v1_genomic.fna
GCA_900184685.1_PRJEB20857_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/LLG/drep/drep/dereplicated_genomes/GCA_900184685.1_PRJEB20857_genomic.fna


In [22]:
# GTDBTk summary
tax = file.path(work_dir, 'LLG', 'gtdbtk', 'gtdbtk_bac_summary.tsv') %>%
    read.delim(, sep='\t') %>%
    separate(classification, 
             c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'),
             sep=';') %>%
    select(-note, -classification_method, -pplacer_taxonomy,
           -other_related_references.genome_id.species_name.radius.ANI.AF.)
tax

user_genome,Domain,Phylum,Class,Order,Family,Genus,Species,fastani_reference,fastani_reference_radius,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<dbl>,⋯,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<int>,<fct>,<fct>
GCA_000379665.1_ASM37966v1_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Marinifilaceae,g__Butyricimonas,s__Butyricimonas synergistica,GCF_000379665.1,95,⋯,1.00,GCF_000379665.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas synergistica,100.00,1.00,96.57,11,N/A,N/A
GCA_001915615.1_ASM191561v1_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Marinifilaceae,g__Butyricimonas,s__Butyricimonas faecihominis,GCF_003851945.2,95,⋯,0.78,GCF_003851945.2,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas faecihominis,98.09,0.78,84.37,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_900184685.1_PRJEB20857_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Marinifilaceae,g__Butyricimonas,s__Butyricimonas sp900184685,GCF_900184685.1,95,⋯,1.00,GCF_900184685.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas sp900184685,100.00,1.00,96.71,11,N/A,N/A
GCA_902364335.1_MGYG-HGUT-00203_genomic,d__Bacteria,p__Bacteroidota,c__Bacteroidia,o__Bacteroidales,f__Marinifilaceae,g__Butyricimonas,s__Butyricimonas virosa,GCF_000519105.1,95,⋯,0.85,GCF_000519105.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas virosa,97.98,0.85,96.79,11,N/A,N/A


In [23]:
# checking overlap
cat('-- drep --\n')
overlap(basename(as.character(drep_gen$Fasta)), 
        basename(as.character(drep$genome)))
cat('-- checkm --\n')
overlap(drep$Bin.Id, checkm$Bin.Id)
cat('-- gtdbtk --\n')
overlap(drep$Bin.Id, tax$user_genome)

-- drep --
intersect(x,y): 17 
setdiff(x,y): 0 
setdiff(y,x): 12 
union(x,y): 29 
-- checkm --
intersect(x,y): 29 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 29 
-- gtdbtk --
intersect(x,y): 29 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 29 


In [24]:
# joining based on Bin.Id
drep = drep %>%
    inner_join(checkm, c('Bin.Id')) %>%
    mutate(GEN = genome %>% as.character %>% basename) %>%
    inner_join(drep_gen %>% mutate(GEN = Fasta %>% as.character %>% basename),
               by=c('GEN')) %>%
    inner_join(tax, c('Bin.Id'='user_genome')) #%>%
drep

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000379665.1_ASM37966v1_genomic.fna,100.00,0,GCA_000379665.1_ASM37966v1_genomic,k__Bacteria (UID2569),434,278,186,100.00,0,⋯,1.00,GCF_000379665.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas synergistica,100.00,1.00,96.57,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_002320175.1_ASM232017v1_genomic.fna,98.39,0,GCA_002320175.1_ASM232017v1_genomic,k__Bacteria (UID2569),434,278,186,98.39,0,⋯,0.87,GCF_000519105.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas virosa,98.27,0.87,94.37,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_014287455.1_ASM1428745v1_genomic.fna,100,0,GCA_014287455.1_ASM1428745v1_genomic,k__Bacteria (UID2569),434,278,186,100,0,⋯,0.81,GCF_900184685.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas sp900184685,97.68,0.81,96.73,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900184685.1_PRJEB20857_genomic.fna,100,0,GCA_900184685.1_PRJEB20857_genomic,k__Bacteria (UID2569),434,278,186,100,0,⋯,1.00,GCF_900184685.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas sp900184685,100.00,1.00,96.71,11,N/A,N/A


In [25]:
# filtering by quality
hq_genomes = drep %>%
    filter(completeness >= 90,
           contamination < 5,
           Strain.heterogeneity < 50)
hq_genomes

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000379665.1_ASM37966v1_genomic.fna,100.00,0,GCA_000379665.1_ASM37966v1_genomic,k__Bacteria (UID2569),434,278,186,100.00,0,⋯,1.00,GCF_000379665.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas synergistica,100.00,1.00,96.57,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_002320175.1_ASM232017v1_genomic.fna,98.39,0,GCA_002320175.1_ASM232017v1_genomic,k__Bacteria (UID2569),434,278,186,98.39,0,⋯,0.87,GCF_000519105.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas virosa,98.27,0.87,94.37,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_014287455.1_ASM1428745v1_genomic.fna,100,0,GCA_014287455.1_ASM1428745v1_genomic,k__Bacteria (UID2569),434,278,186,100,0,⋯,0.81,GCF_900184685.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas sp900184685,97.68,0.81,96.73,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900184685.1_PRJEB20857_genomic.fna,100,0,GCA_900184685.1_PRJEB20857_genomic,k__Bacteria (UID2569),434,278,186,100,0,⋯,1.00,GCF_900184685.1,95,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Marinifilaceae;g__Butyricimonas;s__Butyricimonas sp900184685,100.00,1.00,96.71,11,N/A,N/A


In [26]:
# summarizing the taxonomy
df.dims(20)
hq_genomes %>%
    group_by(Family, Genus) %>%
    summarize(n_genomes = n(), .groups='drop')
df.dims()

Family,Genus,n_genomes
<chr>,<chr>,<int>
f__Marinifilaceae,g__Butyricimonas,17


In [27]:
# writing samples table for LLPRIMER
outfile = file.path(work_dir, 'samples_genomes_hq.txt')
hq_genomes %>%
    select(Bin.Id, Fasta) %>%
    rename('Taxon' = Bin.Id) %>%
    mutate(Taxon = gsub('_chromosome.+', '', Taxon),
           Taxon = gsub('_bin_.+', '', Taxon),
           Taxon = gsub('_genomic', '', Taxon),
           Taxon = gsub('_annotated_assembly', '', Taxon),
           Taxid = taxid,
           Domain = domain) %>%
    write_table(outfile)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Butyricimonas/samples_genomes_hq.txt 


# Primer design

### Config

In [28]:
F = file.path(work_dir, 'primers', 'config.yaml')
cat_file(F)

#-- I/O --#
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/samples_genomes_hq.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/primers/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

#-- software parameters --#
# See the README for a description
params:
  ionice: -c 3
  cgp:
    prokka: ""
    mmseqs:
      method: cluster    # or linclust (faster)
      run: --min-seq-id 0.8 -c 0.8
    vsearch: --id 0.94
    core_genes:
      cds: --perc-genomes-cds 100 --copies-per-genome-cds 1 --max-clusters-cds 500
      rRNA: --perc-genomes-rrna 100 --copies-per-genome-rrna 10 --max-clusters-rrna 500
    align:
      method: linsi
      params: --auto --maxiterate 1000
    primer3:
      consensus: --consensus-threshold 0.34
      number: --num-raw-primers 1000 --num-final-primers 10
     

### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llprimer
$ screen -L -S llprimer-But ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3/Butyricimonas/primers/config.yaml 50 -F --notemp
```

## Summary

### Primers

In [29]:
primer_info = read.delim(file.path(work_dir, 'primers', 'cgp', 'primers_final_info.tsv'), sep='\t')
primer_info %>% unique_n('primers', primer_set)
primer_info

No. of unique primers: 991 


gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd
<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cds,1,286,201,201,0,286f,PRIMER_LEFT,TRCGYTCGKCTYTGGTDCCG,20,⋯,445,465,64.26734,2.827061,61.66667,5.527708,28.84957,27.433750,3.172741,15.30954
cds,1,286,201,201,0,286r,PRIMER_RIGHT,TVCGYAACCAWGCCAAHGCC,20,⋯,626,646,63.56097,2.428073,57.50000,4.166667,42.82996,6.396477,-16.432855,22.12949
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,500,874,117,117,0,874f,PRIMER_LEFT,ATWGAYGCSGGRCCYGAYTT,20,⋯,225,245,62.45535,2.487909,55.00000,5.000000,23.95850,24.20188,23.27590,16.75870
cds,500,874,117,117,0,874r,PRIMER_RIGHT,YCGWATATCRTCCARKCCYCCG,22,⋯,320,342,62.67829,2.221396,56.81818,5.081973,19.97123,19.99833,-26.80071,22.36798


### Gene clusters

In [30]:
# target gene info
gene_annot = read.delim(file.path(work_dir, 'primers', 'cgp', 'core_clusters_info.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_annot %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of clusters', ., '\n')
gene_annot

No. of clusters 484 


gene_type,cluster_id,seq_uuid,seq_orig_name,contig_id,taxon,start,end,score,strand,annotation,cluster_name,clust_id
<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>
cds,1,ba6e70d9efab4a67a21364b8d0c36599,APGDNBDC_03747,KB903373.1,GCA_000379665.1_ASM37966v1,134880,135569,.,+,hypothetical protein,566ebf93955f48b194569f670baaa7b5,1
cds,1,1db21bc2cb8349a58f0c7bed9cb8e7d7,DINPFHMG_00394,DCJY01000025.1,GCA_002320175.1_ASM232017v1,207,947,.,-,hypothetical protein,566ebf93955f48b194569f670baaa7b5,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,500,4d4a337be6aa4cdf8e30cbfc38b85254,DGJGCJFC_01250,JACOOH010000002.1,GCA_014287455.1_ASM1428745v1,624572,625396,.,-,Phosphoribosyl 1%2C2-cyclic phosphate phosphodiesterase,f8df3fd36e314e6f995d3bb8d84fe5ef,500
cds,500,5c0e97d5e7e04f2b917dd10c4a233d61,HIBIFNEK_00668,FXXL01000007.1,GCA_900184685.1_PRJEB20857,762197,763021,.,-,Phosphoribosyl 1%2C2-cyclic phosphate phosphodiesterase,f8df3fd36e314e6f995d3bb8d84fe5ef,500


### Gene cluster annotations

In [31]:
# non-target cds gene annotations
gene_nontarget = read.delim(file.path(work_dir, 'primers', 'cgp', 'nontarget', 'cds_blastx.tsv'), 
                        sep='\t') %>%
    semi_join(primer_info, c('cluster_id')) 
gene_nontarget

cluster_id,query,subject,subject_name,pident,length,mismatch,qstart,qend,sstart,send,evalue,slen,qlen,sscinames,staxids,pident_rank
<int>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<fct>,<fct>,<int>
1,566ebf93955f48b194569f670baaa7b5,WP_118714828.1,zinc metallopeptidase [Odoribacter sp. AF15-53],94.323,229,13,100,786,1,229,2.25e-131,229,789,Odoribacter sp. AF15-53,2292236,1
1,566ebf93955f48b194569f670baaa7b5,RGG45931.1,None,94.323,229,13,100,786,1,229,2.15e-130,229,789,Odoribacter sp. AM16-33;Odoribacter sp. AF21-41,2292048;2293111,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
500,f8df3fd36e314e6f995d3bb8d84fe5ef,WP_055130916.1,MBL fold metallo-hydrolase [Pedobacter sp. Hv1],54.150,253,116,1,759,1,253,1.39e-104,254,762,Pedobacter sp. Hv1,1740090,30
500,f8df3fd36e314e6f995d3bb8d84fe5ef,WP_110360170.1,MBL fold metallo-hydrolase [Marinifilum breve],55.159,252,113,1,756,2,253,1.50e-104,254,762,Marinifilum breve,2184082,26


### Cluster taxonomic uniqueness

In [32]:
# closest seqID
gene_nontarget %>%
    filter(pident_rank == 1) %>%
    .$pident %>% summary_x('seq-ID to closest non-target')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
seq-ID to closest non-target,81.107,92.44225,96.508,94.98219,98.60025,99.813,6.794,2.774


### Primer quality

In [33]:
# summary
primer_info %>% unique_n('primer sets', primer_set)
primer_info %>% distinct(gene_type, cluster_id) %>% nrow %>% cat('No. of unique clusters:', ., '\n')

No. of unique primer sets: 991 
No. of unique clusters: 484 


In [34]:
# primers per cluster
primer_info %>%
    distinct(cluster_id, primer_set) %>%
    group_by(cluster_id) %>%
    summarize(n_primer_sets = n(), .groups='drop') %>%
    .$n_primer_sets %>% summary_x('primer sets per cluster')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
primer sets per cluster,1,10,10,9.615702,10,10,3.646,1.489


In [35]:
# primer quality
primer_info %>% filter(primer_type == 'PRIMER_LEFT') %>% .$amplicon_size_avg %>% summary_x('mean amplicon size')
primer_info %>% .$degeneracy %>% summary_x('degeneracy')
primer_info %>% .$degeneracy_3prime %>% summary_x('degeneracy (3-prime)')

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
mean amplicon size,94.35294,127,164,167.3896,204,250,55.029,22.466


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy,1,8,16,21.48765,24,128,47.279,19.302


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
degeneracy (3-prime),1,1,2,1.905995,2,6,1.866,0.762


In [36]:
# adding non-target pident info
tmp1 = gene_nontarget %>%
    filter(pident_rank == 1) %>%
    distinct(cluster_id, pident, sscinames) %>%
    mutate(gene_type = 'cds')

primer_info_j = primer_info %>%
    inner_join(tmp1, c('gene_type', 'cluster_id'))
primer_info_j

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,⋯,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
cds,1,286,201,201,0,286f,PRIMER_LEFT,TRCGYTCGKCTYTGGTDCCG,20,⋯,64.26734,2.827061,61.66667,5.527708,28.84957,27.43375,3.172741,15.30954,94.323,Odoribacter sp. AF15-53
cds,1,286,201,201,0,286f,PRIMER_LEFT,TRCGYTCGKCTYTGGTDCCG,20,⋯,64.26734,2.827061,61.66667,5.527708,28.84957,27.43375,3.172741,15.30954,94.323,Odoribacter sp. AM16-33;Odoribacter sp. AF21-41
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
cds,500,874,117,117,0,874f,PRIMER_LEFT,ATWGAYGCSGGRCCYGAYTT,20,⋯,62.45535,2.487909,55.00000,5.000000,23.95850,24.20188,23.27590,16.75870,93.281,Odoribacter sp. AM16-33;Odoribacter sp. AF21-41
cds,500,874,117,117,0,874r,PRIMER_RIGHT,YCGWATATCRTCCARKCCYCCG,22,⋯,62.67829,2.221396,56.81818,5.081973,19.97123,19.99833,-26.80071,22.36798,93.281,Odoribacter sp. AM16-33;Odoribacter sp. AF21-41


In [37]:
# arrange by "best" primers
primer_info_jf = primer_info_j %>%
    group_by(primer_set) %>%
    mutate(max_degeneracy_3prime = max(degeneracy_3prime),  # max of [Fwd,Rev]
           max_degeneracy = max(degeneracy)) %>%
    ungroup() %>%
    arrange(pident, max_degeneracy_3prime, max_degeneracy) %>%
    head(n=30)

df.dims(30,40)
primer_info_jf
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>
cds,469,677,122,122,0,677f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,2,64
cds,469,677,122,122,0,677r,PRIMER_RIGHT,CCBGGGGGATAYTGYGCCA,19,12,2,910,929,63.99539,2.171444,66.66667,4.472824,42.790866,2.5680978,3.518264,11.0215662,81.107,Odoribacter sp. AF15-53,2,64
cds,469,84,122,122,0,84f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,3,12
cds,469,84,122,122,0,84r,PRIMER_RIGHT,CCBGGGGGATAYTGYGCCAT,20,12,1,909,929,64.24558,2.069738,63.33333,4.249183,42.928759,2.8664437,3.518264,11.0215662,81.107,Odoribacter sp. AF15-53,3,12
cds,469,700,119,119,0,700f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,3,36
cds,469,700,119,119,0,700r,PRIMER_RIGHT,GGGGGATAYTGYGCCATSGT,20,8,2,906,926,62.86230,1.692288,60.00000,3.535534,41.156774,0.1051210,8.175715,7.8528329,81.107,Odoribacter sp. AF15-53,3,36
cds,469,720,119,119,0,720f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,4,16
cds,469,720,119,119,0,720r,PRIMER_RIGHT,GGGGGATAYTGYGCCATSGTY,21,16,4,905,926,63.52770,1.607231,59.52381,4.123930,41.156774,0.1051210,8.175715,7.8528329,81.107,Odoribacter sp. AF15-53,4,16
cds,469,308,118,118,0,308f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,4,24


#### w/ gene annotations

In [38]:
# joining
primer_info_jfj = primer_info_jf %>%
    left_join(gene_annot %>% distinct(gene_type, cluster_id, annotation),
              c('gene_type', 'cluster_id'))

df.dims(30,50)
primer_info_jfj
df.dims()

gene_type,cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,degeneracy_3prime,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd,pident,sscinames,max_degeneracy_3prime,max_degeneracy,annotation
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<int>,<int>,<fct>
cds,469,677,122,122,0,677f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,2,64,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,677,122,122,0,677r,PRIMER_RIGHT,CCBGGGGGATAYTGYGCCA,19,12,2,910,929,63.99539,2.171444,66.66667,4.472824,42.790866,2.5680978,3.518264,11.0215662,81.107,Odoribacter sp. AF15-53,2,64,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,84,122,122,0,84f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,3,12,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,84,122,122,0,84r,PRIMER_RIGHT,CCBGGGGGATAYTGYGCCAT,20,12,1,909,929,64.24558,2.069738,63.33333,4.249183,42.928759,2.8664437,3.518264,11.0215662,81.107,Odoribacter sp. AF15-53,3,12,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,700,119,119,0,700f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,3,36,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,700,119,119,0,700r,PRIMER_RIGHT,GGGGGATAYTGYGCCATSGT,20,8,2,906,926,62.86230,1.692288,60.00000,3.535534,41.156774,0.1051210,8.175715,7.8528329,81.107,Odoribacter sp. AF15-53,3,36,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,720,119,119,0,720f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,4,16,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,720,119,119,0,720r,PRIMER_RIGHT,GGGGGATAYTGYGCCATSGTY,21,16,4,905,926,63.52770,1.607231,59.52381,4.123930,41.156774,0.1051210,8.175715,7.8528329,81.107,Odoribacter sp. AF15-53,4,16,Peptidoglycan D%2CD-transpeptidase MrdA
cds,469,308,118,118,0,308f,PRIMER_LEFT,AGTCCSGGDTATGAYCCGCA,20,12,1,807,827,63.37921,1.585835,59.16667,3.435921,54.529677,10.4477986,12.546118,13.3155804,81.107,Odoribacter sp. AF15-53,4,24,Peptidoglycan D%2CD-transpeptidase MrdA


#### Writing table

In [39]:
F = file.path(work_dir, 'best_primers.tsv')
write_table(primer_info_jfj, F)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/v0.3//Butyricimonas/best_primers.tsv 


# sessionInfo

In [40]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llg/')

LLG
===

Ley Lab Genome analysis pipeline (LLG)

* Version: 0.2.4
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/gtdbtk.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- pigz
- conda-forge::networkx
- bioconda::gtdbtk=1.4

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/checkm.yaml <==
channels:
- bioconda
dependencies:
- python=2.7
- pigz
- bioconda::prodigal
- bioconda::pplacer
- bioconda::checkm-genome

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/quast.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::seqkit
- bioconda::quast>=5.0.0

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/sourmash.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::sourmash=2.0.0a4

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/fastqc.yaml <==
channel

In [41]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/')

LLPRIMER

Ley Lab Primer generation pipeline (LLPRIMER)

* Version: 0.3.0
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/pdp.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- python=3.7
- intervaltree
- prodigal
- blast
- bedtools
- mafft
- mummer=3.23
- emboss
- primer3=1.1.4
- biopython<1.78
- pybedtools
- joblib
- tqdm
- openpyxl

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/genes.yaml <==
channels:
- bioconda
dependencies:
- pigz
- python=3
- numpy
- pyfaidx
- bioconda::seqkit
- bioconda::fasta-splitter
- bioconda::vsearch
- bioconda::mmseqs2
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/aln.yaml <==
channels:
- bioconda
- conda-forge
dependencies:
- pigz
- bioconda::revtrans
- bioconda::kalign3
- bioconda::mafft

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/env

In [42]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ggplot2_3.3.1     tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6     magrittr_1.5     munsell_0.5.0    tidyselect_1.1.0
 [5] uuid_0.1-4       colorspace_1.4-1 R6_2.4.1         rlang_0.4.6     
 [9] tools_3.6.3